## imports

In [32]:
import matplotlib
matplotlib.use('TKagg')
import matplotlib.pyplot as plt
from matplotlib.pyplot import *
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import numpy as np
from scipy.spatial import Delaunay
import math
import random

## fonctions de dessins

In [33]:
def draw_diagonals(polygon, diagonals):
    # Draw the polygon
    plt.figure(figsize=(8,8))
    plt.gca().set_aspect('equal')
    x, y = zip(*polygon)
    plt.plot(x + (x[0],), y + (y[0],), '-o')

    # Draw the diagonals
    for i, j in diagonals:
        x, y = zip(polygon[i], polygon[j])
        plt.plot(x, y, 'r')
    
    # Show the plot
    plt.show()


In [34]:
def draw_polygon_and_triangles(points, triangles, zoom_out=1):
    # Créer la figure et l'axe
    fig, ax = plt.subplots()

    # Dessiner le polygone
    poly = Polygon(points, facecolor='white', edgecolor='black')
    ax.add_patch(poly)

    # Dessiner chaque triangle avec une couleur aléatoire
    for triangle in triangles:
        color = (random.random(), random.random(), random.random())
        tri = Polygon(triangle, facecolor=color, edgecolor='black')
        ax.add_patch(tri)

    # Définir les limites de l'axe
    margin = max(1, max(abs(point[0]) for point in points), max(abs(point[1]) for point in points))
    ax.set_xlim([-margin*zoom_out, margin*zoom_out])
    ax.set_ylim([-margin*zoom_out, margin*zoom_out])

    # Afficher la figure
    plt.show()

## Essais Successifs

##### Fonction pour vérifier si une corde est valide La fonction `validecorde_exact` prend en entrée une corde `corde` à tester, une liste de cordes déjà dessinées `drawn_chords` et le nombre de sommets `n` du cycle. Elle renvoie `True` si la corde est valide et `False` sinon.
##### La fonction vérifie si la corde relie deux sommets différents, si elle n'est pas déjà dessinée, si elle ne relie pas des sommets adjacents et si elle ne croise aucune corde déjà dessinée.


In [35]:
def validecorde_exact(corde, drawn_chords,n):
    (i,j) = corde
    # On récupère les indices des deux sommets de la corde
    if i==j :
        return False
    # Si i et j sont égaux, alors la corde n'est pas valide car elle relie un sommet à lui-même
    if (i,j) in drawn_chords or (j,i) in drawn_chords:
        return False
    # Vérifier si la corde i,j est déjà dessinée
    if i == (j+1)%n or j == (i+1)%n : 
        return False
    # Si i et j sont adjacents, alors la corde n'est pas valide car elle relie des sommets voisins
    # On utilise le modulo n pour considérer le cas où j=i+1=n ou i=j+1=n

    # Vérifier s'il y a une corde entre i,j qui croise une corde déjà dessinée
    for (k, p) in drawn_chords:
        if (k < i and i < p < j) or (k > i and i > p > j):
            return False
    # On vérifie si la corde (i,j) croise une corde déjà dessinée, 
    # en testant si les sommets k et p d'une corde dessinée (k,p) sont tels que 
    # l'un des deux soit strictement entre i et j dans l'ordre des sommets du cycle, c'est-à-dire k<i<j<p ou p<i<j<k.

    return True
    # Si on arrive jusqu'ici, alors la corde (i,j) ne croise aucune corde déjà dessinée et est donc valide.

#### Essais Successifs
#####
##### La section de code suivante utilise une approche "essais successifs" pour trouver un ensemble valide de diagonales qui peut être utilisé pour trianguler un polygone convexe. L'algorithme commence par choisir un sommet arbitraire et essaie ensuite de le connecter à chaque autre sommet du polygone, un à la fois, pour voir si la diagonale résultante est valide. Une diagonale est considérée "valide" si elle n'intersecte aucune des diagonales précédemment ajoutées.
##### Si une diagonale valide est trouvée, elle est ajoutée à la liste des diagonales et l'algorithme passe au sommet suivant. Si aucune diagonale valide ne peut être trouvée pour un sommet donné, l'algorithme "recule" et essaie le sommet précédent à nouveau, en supprimant la dernière diagonale ajoutée. Cela continue jusqu'à ce qu'un ensemble valide de diagonales soit trouvé qui peut être utilisé pour trianguler l'ensemble du polygone.
##### Bien que cette approche ne garantisse pas de trouver un ensemble valide de diagonales dans tous les cas, elle fonctionne bien pour de nombreux polygones convexes et est relativement simple à mettre en œuvre.


In [36]:
def validecorde(corde, drawn_chords):
    (i,j) = corde
    # On récupère les indices des deux sommets de la corde
    if i==j :
        return False
    # Si i et j sont égaux, alors la corde n'est pas valide car elle relie un sommet à lui-même
    if (i,j) in drawn_chords:
        return False
    # Vérifier si la corde i,j est déjà dessinée
    
    # Vérifier s'il y a une corde entre i,j qui croise une corde déjà dessinée
    for (k, p) in drawn_chords:
        if k < i and i < p < j:
            return False
        if i < k and k < j < p:
            return False
    # On vérifie si la corde (i,j) croise une corde déjà dessinée, 
    # en testant si les sommets k et p d'une corde dessinée (k,p) sont tels que 
    # l'un des deux soit strictement entre i et j dans l'ordre des sommets du cycle, c'est-à-dire k<i<j<p ou i<k<j<p.

    return True
    # Si on arrive jusqu'ici, alors la corde (i,j) ne croise aucune corde déjà dessinée et est donc valide.

def triangulate_convex_polygon(C):
    diagonals = []
    n = len(C)
    i = n-1
    j = 0
    blacklist = set()

    while len(diagonals) < n-2:
        if validecorde((i, j), diagonals) and (i,j) not in blacklist:
            diagonals.append((i, j))
            j = (j+1) % n
        else:
            i = (i-1) % n
            blacklist.add(diagonals[-1])
            diagonals.pop()
    # On parcourt les paires de sommets (i,j) de C dans l'ordre (i=n-1,j=0), (i=n-2,j=0), ..., (i=1,j=0), (i=0,j=1), 
    # et on ajoute à la liste diagonals les diagonales qui peuvent être tracées selon la méthode du balayage de gauche à droite. 
    # Si la diagonale (i,j) est invalide ou a déjà été ajoutée à la liste, on revient en arrière pour essayer la diagonale précédente. 
    # On ajoute les diagonales invalides à la liste blacklist pour ne pas les retester.

    return diagonals
    # La liste diagonals contient toutes les diagonales tracées et forme une triangulation de C.

#### test

In [37]:
C = [(-1, 1), (-1, 3), (1, 4), (3, 3), (2, 1), (0, 0)]
diagonals = triangulate_convex_polygon(C)
draw_diagonals(C, diagonals)

## Dynamique

##### L'algorithme de triangulation de polygones suivant est basé sur la méthode de programmation dynamique. Il consiste à trouver des triangles formés par les sommets du polygone de manière à le découper en triangles non intersectants. Pour cela, l'algorithme cherche des "oreilles" du polygone, c'est-à-dire des triangles formés par trois sommets qui ne contiennent pas d'autres sommets du polygone à l'intérieur. Une fois une oreille trouvée, elle est enlevée du polygone et le processus est répété sur le polygone restant jusqu'à ce qu'il ne reste plus de sommets. L'algorithme retourne alors la liste des triangles formés. 


In [38]:
def triangulate_polygon(polygon):
    triangles = []
    n = len(polygon)
    
    # Si le nombre de sommets du polygone est inférieur à 3, il ne peut pas être triangulé
    if n < 3:
        return triangles
    
    # Parcourt tous les sommets du polygone
    for i in range(n):
        prev = (i - 1) % n
        curr = i
        next = (i + 1) % n
        
        # Vérifie si le triangle formé par les sommets i, i+1 et i+2 est une "oreille" du polygone
        if is_ear(polygon, prev, curr, next):
            # Ajoute le triangle à la liste des triangles et enlève le sommet i+1 du polygone
            triangles.append((polygon[prev], polygon[curr], polygon[next]))
            polygon.pop(curr)
            # Répète le processus de triangulation sur le polygone restant
            return triangles + triangulate_polygon(polygon)
    
    # Si aucun triangle n'a été trouvé pour trianguler le polygone, la triangulation est terminée
    return triangles

def is_ear(polygon, i, j, k):
    # Vérifie si le triangle formé par les sommets i, j et k ne contient pas d'autres sommets du polygone
    for m in range(len(polygon)):
        if m != i and m != j and m != k and is_inside_triangle(polygon[m], polygon[i], polygon[j], polygon[k]):
            return False
    return True

def is_inside_triangle(p, a, b, c):
    # Vérifie si le point p est à l'intérieur du triangle formé par les sommets a, b et c
    return is_same_side(p, a, b, c) and is_same_side(p, b, a, c) and is_same_side(p, c, a, b)

def is_same_side(p1, p2, a, b):
    # Vérifie si les points p1 et p2 sont du même côté de la ligne formée par les sommets a et b
    cp1 = cross_product(b[0] - a[0], b[1] - a[1], p1[0] - a[0], p1[1] - a[1])
    cp2 = cross_product(b[0] - a[0], b[1] - a[1], p2[0] - a[0], p2[1] - a[1])
    return cp1 * cp2 >= 0

def cross_product(x1, y1, x2, y2):
    # Calcule le produit vectoriel de deux vecteurs
    return x1 * y2 - x2 * y1

#### test

In [39]:
C = [
(180.49111067193468, 494.6643076028935)
,(153.57298273821428, 491.8990116382506)
,(124.90587499022688, 482.56823390821756)
,(94.8304013738639, 465.4617906982039)
,(79.65802095744523, 451.0048215906153)
,(58.657721967764864, 430.7563139713908)
,(22.125366180142493, 377.135118374759)
,(15.018875463509772, 340.6585145248268)
,(7.423964246997095, 237.6564116158071)
,(13.25404024697252, 177.78175687888677)
,(24.105947793609307, 125.83176826111064)
,(42.27962105864147, 78.94556462003533)
,(55.46466046939314, 46.68742678288962)
,(65.52681116466663, 33.29498410064791)
,(126.6137140781215, 23.464199508610008)
,(282.78977610022685, 9.617583587439471)
,(356.9265394776163, 7.778598135969372)
,(363.79596068265454, 9.146574531481711)
,(363.8598081309624, 9.168552444235868)
,(410.2550243605333, 28.412895732593867)
,(446.6594395588621, 47.50671199284218)
,(460.78665921087867, 64.044003818305)
,(475.03239783670716, 92.81868127186299)
,(465.8151124650581, 164.52408991617017)
,(454.9461041438435, 219.66632833107658)
,(327.8984491216154, 442.4023448567188)
,(285.1837042223283, 490.3952039723944)
,(281.86092750835576, 490.7914741816579)
,(276.3170853336246, 491.370066028904)]
triangles = triangulate_polygon(C)
draw_polygon_and_triangles(C, triangles)

## Greedy

##### Ce code implémente une méthode de triangulation d'un polygone convexe en utilisant une méthode de programmation gloutonne. L'idée est de choisir la corde extérieure la plus courte à chaque étape de la triangulation.

##### La fonction `triangle_greedy` prend en entrée une liste de points qui représentent les sommets du polygone convexe. Elle retourne une liste de triangles, chaque triangle étant représenté par un tuple de 3 points. 

In [40]:
def line_length(p1, p2):
    """
    Calculates the Euclidean distance between two 2D points p1 and p2.
    Returns a float.
    """
    x1, y1 = p1
    x2, y2 = p2
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

In [41]:
def triangle_greedy(points):
    # Vérifier que les points forment un polygone convexe
    n = len(points)
    if n < 3:
        # Si la liste de points contient moins de 3 points, elle ne peut pas former un polygone convexe,
        # donc on retourne une liste vide
        return []

    # Initialiser la liste des triangles
    triangles = []

    # Tant qu'il reste des sommets à trianguler
    while n > 2:
        # Rechercher la corde extérieure la plus courte
        min_distance = float('inf')
        min_index = -1
        for i in range(n):
            p1 = points[i]
            p2 = points[(i+1)%n]
            for j in range(1, n-1):
                if j == i or j == (i+1)%n:
                    continue
                p3 = points[(i+j)%n]
                p4 = points[(i+j+1)%n]
                distance = line_length(p1, p3) + line_length(p2, p4)
                if distance < min_distance:
                    min_distance = distance
                    min_index = (i+j)%n

        # Ajouter le triangle formé par la corde extérieure
        triangles.append((points[min_index], points[(min_index+1)%n], points[(min_index-1)%n]))

        # Retirer le point sélectionné
        points.pop(min_index)
        n -= 1

    # Retourner la liste des triangles
    return triangles

#### test

In [42]:
C = [
(180.49111067193468, 494.6643076028935)
,(153.57298273821428, 491.8990116382506)
,(124.90587499022688, 482.56823390821756)
,(94.8304013738639, 465.4617906982039)
,(79.65802095744523, 451.0048215906153)
,(58.657721967764864, 430.7563139713908)
,(22.125366180142493, 377.135118374759)
,(15.018875463509772, 340.6585145248268)
,(7.423964246997095, 237.6564116158071)
,(13.25404024697252, 177.78175687888677)
,(24.105947793609307, 125.83176826111064)
,(42.27962105864147, 78.94556462003533)
,(55.46466046939314, 46.68742678288962)
,(65.52681116466663, 33.29498410064791)
,(126.6137140781215, 23.464199508610008)
,(282.78977610022685, 9.617583587439471)
,(356.9265394776163, 7.778598135969372)
,(363.79596068265454, 9.146574531481711)
,(363.8598081309624, 9.168552444235868)
,(410.2550243605333, 28.412895732593867)
,(446.6594395588621, 47.50671199284218)
,(460.78665921087867, 64.044003818305)
,(475.03239783670716, 92.81868127186299)
,(465.8151124650581, 164.52408991617017)
,(454.9461041438435, 219.66632833107658)
,(327.8984491216154, 442.4023448567188)
,(285.1837042223283, 490.3952039723944)
,(281.86092750835576, 490.7914741816579)
,(276.3170853336246, 491.370066028904)]
triangles = triangle_greedy(C)
draw_polygon_and_triangles(C, triangles)